In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

import pandas as pd
import json

import geopy
from geopy import distance
from geopy.distance import vincenty
# from tqdm import tqdm
# tqdm.pandas()
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
from scipy import stats
from dateutil import parser
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
import numpy as np

import eli5
from eli5.sklearn import PermutationImportance
# from skopt.space import Real
import matplotlib.colors as clt
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
# from tqdm import tqdm
from scipy.stats import kurtosis, skew, moment
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR
from sklearn.metrics import mean_absolute_error
%matplotlib inline
from keras.layers import Dense
# Change pandas viewing options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zipfile
from keras.models import Sequential
# zf = zipfile.ZipFile('train.csv') # having First.csv zipped file.
# df = pd.read_csv(zf.open('First.csv'))
# zf.namelist()

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [5]:
def gen_features(X):
    strain = []
    strain.append(X.mean())
    strain.append(X.median())
    strain.append(X.std())
    strain.append(X.min())
    strain.append(X.max())
    strain.append(X.kurtosis())
    strain.append(X.skew())
    strain.append(X.ptp())
    strain.append(X.var())
#     strain.append(X.mad())
    strain.append(np.quantile(X, 0.01))
    strain.append(np.quantile(X, 0.05))
    zc=np.fft.fft(X)
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    strain.append(realFFT.mean())
    strain.append(realFFT.std())
    strain.append(realFFT.max())
    strain.append(realFFT.min())
    strain.append(imagFFT.mean())
    strain.append(imagFFT.std())
    strain.append(imagFFT.max())
    strain.append(imagFFT.min())
    strain.append(moment([X], moment=0, axis=1, nan_policy='omit'))
    strain.append(moment([X], moment=1, axis=1, nan_policy='omit'))
    strain.append(moment([X], moment=2, axis=1, nan_policy='omit'))
    strain.append(moment([X], moment=3, axis=1, nan_policy='omit'))
    strain.append(moment([X], moment=4, axis=1, nan_policy='omit'))
    strain.append(moment([X], moment=5, axis=1, nan_policy='omit'))



    strain.append((np.clip([X],0,1000)).mean())
    return pd.Series(strain)

# final.describe()
    

In [6]:
train = pd.read_csv('train.csv.zip', iterator=True, chunksize=200_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
# train=pd.read_csv('../input/train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
#

In [ ]:

X_train1 = pd.DataFrame()
y_train1= pd.Series()
for df in train:
    df=df.drop(df[df.acoustic_data >400].index)

    ch = gen_features(df['acoustic_data'])
    X_train1 = X_train1.append(ch, ignore_index=True)
    y_train1 = y_train1.append(pd.Series(df['time_to_failure'].values[-1]))

In [ ]:
# X_train1.head()

In [ ]:
# X_train1.ix[:,19].min()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, train_size=0.9, random_state=34)

In [ ]:
# train_pool = Pool(X_train_scaled, y_binary)
model=CatBoostRegressor(iterations=1000, depth=3, learning_rate=0.1, loss_function='MAE',boosting_type='Plain')
model.fit(X_train, y_train, silent=True)


In [ ]:
model.best_score_

In [ ]:

y_pred=(model.predict(X_test))
mean_absolute_error(y_test, y_pred, sample_weight=None)

In [ ]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame()

# prepare test data
for seg_id in submission.index:
    seg = pd.read_csv('test/' + seg_id + '.csv')

    ch = gen_features(seg['acoustic_data'])
    X_test = X_test.append(ch, ignore_index=True)

# X_test = scaler.transform(X_test)

In [ ]:
X_test.head()

In [ ]:
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_hat = model.predict(X_test)

In [ ]:
submission['time_to_failure'] = y_hat
submission.to_csv('submission.csv')

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)

In [ ]:
X_test.columns=['mean','median','std','min','max','kurt','skew','ptp','var','q.01','q.05','Rmean','Rstd','Rmax','Rmin','Imean','Istd','Imax','Imin','0M','1M','2M','3M','4M','5M','thres']


In [ ]:
fold_importance_df=pd.DataFrame()
fold_importance_df['Feature']=X_test.columns
feature_importance_df=pd.DataFrame()
fold_importance_df["importance"]=model.feature_importances_[:len(X_train1.columns)]
feature_importance_df=pd.concat([feature_importance_df, fold_importance_df], axis=0)
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:200].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

# plt.figure(figsize=(14,26))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features (averaged over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')